# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [75]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [76]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [77]:
file_path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list.extend(glob.glob(os.path.join(root,'*')))

In [78]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)

# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

## Print total number of rows 
print(len(full_data_rows_list))

8071


In [79]:
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [80]:
## Test Total No of records in Event data file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for records in f))

6832


## Part II. Apache Cassandra code to answer some basic queries

#### Creating a Cluster

In [81]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [82]:
query_create_keyspace= """
CREATE KEYSPACE IF NOT EXISTS udacity
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor':1}
"""
session.execute(query_create_keyspace)

#### Set Keyspace

In [83]:
session.set_keyspace("udacity")

### Now we need to create tables to run the following queries. With Apache Cassandra you model the database tables on the queries you want to run.

### Writing queries to ask the following three questions from the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




Query 1 uses sessoinId and itemInSession as a composite primary key to return unique rows 

In [84]:
query1 = """SELECT artist, song, length from music_app_history WHERE sessionId=338 AND itemInSession=4"""

In [85]:
drop_statement = """DROP TABLE IF EXISTS music_app_history"""
session.execute(drop_statement)

create_statement = """CREATE TABLE IF NOT EXISTS music_app_history
(artist text, song text, length double, sessionId int, itemInSession int, 
PRIMARY KEY(sessionId, itemInSession))"""

session.execute(create_statement)

In [86]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO music_app_history (artist, song, length, sessionId, itemInSession) """
        query = query + """VALUES(%s, %s, %s, %s, %s)""" 
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Verify that the data have been inserted into each table

In [87]:
test_query = """SELECT artist, song, length from music_app_history where sessionId=139 AND itemInSession=1 """
records = session.execute(test_query)

for rows in records:
    print(rows)

Query 2 uses userId and sessionId as the primary composite key and itemInSession as clustering column for sorting     

In [88]:
query2 = """SELECT artist, song, firstName, lastName from song_by_session WHERE userid=10 AND sessionId=182"""

drop_statement = """DROP TABLE IF EXISTS song_by_session"""
session.execute(drop_statement)

create_statement = """CREATE TABLE IF NOT EXISTS song_by_session (artist text, song text, firstName text, lastName text, userId int, sessionId int, itemInSession int,
PRIMARY KEY((userid, sessionId), itemInSession))"""
session.execute(create_statement)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO song_by_session (artist, song, firstName, lastName, userid, sessionId, itemInSession) """
        query = query + """VALUES(%s, %s, %s, %s, %s, %s, %s)""" 
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))
        

Query 3 uses userId and song as the primary composite key to return unique rows 

In [90]:
query3 = """SELECT firstName, lastName from user_by_song
WHERE song='All Hands Against His Own'"""
                 
drop_statement = """DROP TABLE IF EXISTS user_by_song"""
session.execute(drop_statement)

create_statement = """CREATE TABLE IF NOT EXISTS user_by_song
(firstName text, lastName text, song text, userId int,
PRIMARY KEY(song, userId))"""
session.execute(create_statement)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_by_song (firstName, lastName, song, userId) """
        query = query + """VALUES(%s, %s, %s, %s)""" 
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

In [91]:
test_query = """SELECT firstName, lastName from user_by_song where song='You Gotta Be'"""
records = session.execute(test_query)

for rows in records:
    print(rows) 

Row(firstname='Kaylee', lastname='Summers')


### Dropping the tables before closing out the sessions

In [92]:
drop_statement = """DROP TABLE IF EXISTS music_app_history"""
session.execute(drop_statement)

In [93]:
drop_statement = """DROP TABLE IF EXISTS song_by_session"""
session.execute(drop_statement)

In [94]:
drop_statement = """DROP TABLE IF EXISTS user_by_song"""
session.execute(drop_statement)

### Closing the session and cluster connection¶

In [95]:
session.shutdown()
cluster.shutdown()